<a href="https://colab.research.google.com/github/bladerun16/BBS_ML_repository/blob/main/transactional_to_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformation of a **transactional file** into a matrix of numbers
We will prepare an example data file, then we will show step by step the transformation.

The steps are _incremental_, to show the effects of the _composition_ of the pandas functions.

In a real solution it will be sufficient to use the last cell with the entire composition

In [1]:
import pandas as pd
# prepare an example data file
data = {'TransId': [0,0,0,1,1,2,2,2,2,3,3], 'Item': ['A','C','B','B','A','D','F','B','A','A','F'], 'QTY':[2,1,3,1,2,5,1,4,2,2,1]}
df = pd.DataFrame(data)
df

,TransId,Item,QTY
0,0,A,2
1,0,C,1
2,0,B,3
3,1,B,1
4,1,A,2
5,2,D,5
6,2,F,1
7,2,B,4
8,2,A,2
9,3,A,2


## Group by
- put in the parentheses the list of featured that will generate groups,
- select the features on which you want to make an aggregation (e.g. count, sum, average, ...)
- put at the end the aggretation function

In [2]:
df.groupby(['TransId', 'Item'])['QTY'].sum()

TransId  Item
0        A       2
         B       3
         C       1
1        A       2
         B       1
2        A       2
         B       4
         D       5
         F       1
3        A       2
         F       1
Name: QTY, dtype: int64

## `unstack`
Transfer the distinct values of the innermost grouping feature into columns.

In [3]:
df.groupby(['TransId', 'Item'])['QTY'].sum().unstack()

Item,A,B,C,D,F
TransId,,,,,
0,2.0,3.0,1.0,NaN,NaN
1,2.0,1.0,NaN,NaN,NaN
2,2.0,4.0,NaN,5.0,1.0
3,2.0,NaN,NaN,NaN,1.0


In [ ]:
df.groupby(['TransId', 'Item'])['QTY'].sum().unstack().reset_index()

Item,TransId,A,B,C,D,F
0,0,2.0,3.0,1.0,NaN,NaN
1,1,2.0,1.0,NaN,NaN,NaN
2,2,2.0,4.0,NaN,5.0,1.0
3,3,2.0,NaN,NaN,NaN,1.0


In [ ]:
df.groupby(['TransId', 'Item'])['QTY'].sum().unstack().reset_index().set_index('TransId')

Item,A,B,C,D,F
TransId,,,,,
0,2.0,3.0,1.0,NaN,NaN
1,2.0,1.0,NaN,NaN,NaN
2,2.0,4.0,NaN,5.0,1.0
3,2.0,NaN,NaN,NaN,1.0


In [9]:
df_encoded=df.groupby(['TransId', 'Item'])['QTY'].sum().unstack().reset_index().set_index('TransId').fillna(0)
df_encoded

Item,A,B,C,D,F
TransId,,,,,
0,2.0,3.0,1.0,0.0,0.0
1,2.0,1.0,0.0,0.0,0.0
2,2.0,4.0,0.0,5.0,1.0
3,2.0,0.0,0.0,0.0,1.0


In [10]:
basket_bool=df_encoded.astype('bool')
basket_bool

Item,A,B,C,D,F
TransId,,,,,
0,True,True,True,False,False
1,True,True,False,False,False
2,True,True,False,True,True
3,True,False,False,False,True


In [13]:
from mlxtend.frequent_patterns import apriori, association_rules

In [19]:
min_support = .1
freq_itemsets = apriori(basket_bool, min_support=min_support)
freq_itemsets

,support,itemsets
0,1.00,(0)
1,0.75,(1)
2,0.25,(2)
3,0.25,(3)
4,0.50,(4)
5,0.75,"(0, 1)"
6,0.25,"(0, 2)"
7,0.25,"(0, 3)"
8,0.50,"(0, 4)"
9,0.25,"(1, 2)"


In [21]:
metric = 'confidence'
ar = association_rules(freq_itemsets, metric=metric)
ar.sort_values('lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
29,(3),"(0, 1, 4)",0.25,0.25,0.25,1.0,4.000000,0.1875,inf,1.000000
22,"(0, 1, 4)",(3),0.25,0.25,0.25,1.0,4.000000,0.1875,inf,1.000000
20,(3),"(1, 4)",0.25,0.25,0.25,1.0,4.000000,0.1875,inf,1.000000
18,"(1, 4)",(3),0.25,0.25,0.25,1.0,4.000000,0.1875,inf,1.000000
25,"(0, 3)","(1, 4)",0.25,0.25,0.25,1.0,4.000000,0.1875,inf,1.000000
27,"(1, 4)","(0, 3)",0.25,0.25,0.25,1.0,4.000000,0.1875,inf,1.000000
21,"(0, 1, 3)",(4),0.25,0.50,0.25,1.0,2.000000,0.1250,inf,0.666667
26,"(1, 3)","(0, 4)",0.25,0.50,0.25,1.0,2.000000,0.1250,inf,0.666667
17,"(1, 3)",(4),0.25,0.50,0.25,1.0,2.000000,0.1250,inf,0.666667
16,(3),"(0, 4)",0.25,0.50,0.25,1.0,2.000000,0.1250,inf,0.666667
